# How to crawling?

In [1]:
#import custom module
import havester as hv
import pandas as pd
from IPython.display import clear_output 

hv.IS_LOGGING = False

## 위치을 이용하여 구역 특정

In [2]:
# loc = hv.NLocation(35.18213, 128.1366) # 진주 충무공동
loc = hv.NLocation(37.49911, 127.065463) # 강남 대치동

cur_region = hv.NRegion('', hv.NLocation(37.49911, 127.065463), '0000000000')

# 검색해서 지역 찾기
while True:
    regions = hv.get_region_list(cur_region.no)
    if len(regions) == 0: break
    for region in enumerate(regions): 
        print(region)
        
    idx = int(input("원하는 도시 인덱스(exit -1) : "))
    if idx == -1: break
    cur_region = regions[idx]
    clear_output()
    
sector = hv.get_sector(cur_region.loc)

print(sector)

KeyError: 'regionList'

## 매물 조건 생성

In [94]:
addon = hv.NAddon(
    tradeType=[hv.NAddon.TRADE_DEAL, hv.NAddon.TRADE_LEASE], #목표 거래 - 매매, 전세
    estateType=[hv.NAddon.ESTATE_APT, hv.NAddon.ESTATE_OPST] #목표 매물 - 아파트, 오피스텔
)

## 조건과 구역을 이용하여 매물 탐색

In [95]:
things = [] # 매물 기록
for dirr in hv.NAddon.DIR_EACH: # 모든 방향 (남향 등등)
    addon.direction = [dirr] # 매물 방향 선택
    res = hv.get_things(sector, addon)
    
    for v in res:
        v.dir = dirr # 매물 방향 정의
    things.extend(res)
print('매물 수', len(things))

매물 수 480


## 구역 주변 편의 시설 탐색

In [96]:
neighbors = [] # 편의시설 기록
for nType in hv.NNeighbor.EACH: # 모든 편의시설
    res = hv.get_neighborhood(sector.loc, nType)
    neighbors.extend(res)
    print(nType, len(res))
print('Total', len(neighbors))

BUS 59
METRO 3
INFANT 41
PRESCHOOL 5
SCHOOLPOI 32
HOSPITAL 0
PARKING 77
MART 14
CONVENIENCE 87
WASHING 43
BANK 45
OFFICE 9
Total 415


# 편의시설과 매물 간의 거리 탐색

In [97]:
# 500 == 걸어서 5~10분 이내, 버스로 1분
# 1000 == 버스로 1~2분 

standard = hv.get_distance_standard() # 거리 기준

for (key, value) in standard.items():
    print(key, value)

BUS 500
METRO 500
INFANT 1000
PRESCHOOL 1000
SCHOOLPOI 2000
HOSPITAL 5000
PARKING 1000
MART 1000
CONVENIENCE 350
WASHING 500
BANK 1000
OFFICE 2000


In [98]:
for thing in things: # 매물
    around = hv.NNeighborAround()
    for nei in neighbors: # 편의시설
        d = hv.distance_between(thing.loc, nei.loc)
        if d <= standard[nei.type]: # meter
            around.increase(nei.type)
    thing.neiAround = around

## 시각화

In [99]:
thing_list = []
around_nei = []

for t in things: 
    thing_list.append(t.get_list())
    around_nei.append(t.neiAround.get_list())

thing_df = pd.DataFrame(data=thing_list ,columns=hv.NThing.HEADER)
nei_df = pd.DataFrame(data=around_nei, columns=hv.NNeighborAround.HEADER)
total_df = pd.concat([thing_df, nei_df], axis=1)

total_df.head(5)

,Name,Type,Build,Dir,minArea,maxArea,representativeArea,floorAreaRatio,minDeal,maxDeal,...,INFANT,PRESCHOOL,SCHOOLPOI,HOSPITAL,PARKING,MART,CONVENIENCE,WASHING,BANK,OFFICE
0,대치풍림아이원1차,아파트,200308,EE,94.58,94.58,94.0,0,210000,210000,...,24,4,28,0,54,6,18,14,25,7
1,세영팔래스,아파트,200201,EE,73.77,103.37,103.0,237,175000,200000,...,29,3,32,0,67,12,14,14,35,9
2,삼환SOGOOD(주상복합),아파트,200611,EE,107.38,133.52,133.0,359,220000,220000,...,26,4,30,0,58,10,15,13,36,8
3,대치대우아이빌명문가(주상복합),아파트,200402,EE,41.88,53.58,41.0,637,31000,38000,...,12,3,28,0,49,4,19,10,18,6
4,대치우정에쉐르2(주상복합),아파트,200405,EE,72.18,72.18,72.0,434,99000,130000,...,13,3,23,0,47,5,18,9,18,6


In [100]:
nei_df.describe() # 편의시설 접근도

,BUS,METRO,INFANT,PRESCHOOL,SCHOOLPOI,HOSPITAL,PARKING,MART,CONVENIENCE,WASHING,BANK,OFFICE
count,480.000000,480.000000,480.000000,480.000000,480.000000,480.0,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000
mean,10.333333,0.433333,20.883333,2.966667,28.633333,0.0,45.966667,7.983333,12.583333,10.100000,25.516667,7.750000
std,4.241820,0.559351,7.142081,1.000487,3.709567,0.0,16.072868,3.307443,5.998724,3.935598,9.251741,1.261275
min,2.000000,0.000000,2.000000,0.000000,19.000000,0.0,2.000000,0.000000,1.000000,1.000000,1.000000,6.000000
25%,7.000000,0.000000,15.000000,3.000000,27.000000,0.0,38.750000,5.000000,8.750000,8.000000,19.000000,6.000000
50%,9.500000,0.000000,21.500000,3.000000,29.500000,0.0,49.500000,8.000000,14.000000,10.000000,26.500000,8.000000
75%,14.000000,1.000000,27.250000,4.000000,32.000000,0.0,57.000000,11.000000,18.000000,13.250000,33.250000,9.000000
max,18.000000,2.000000,31.000000,4.000000,32.000000,0.0,70.000000,13.000000,22.000000,19.000000,41.000000,9.000000


## Dataframe 저장

In [101]:
total_df.to_csv('./house_price_data.csv', sep=',', na_rep='NaN')